# Model input ERP format preparation 

In this notebook: 
- Necessary inputs
- Read all epochs
- Function to create dataframe with average mismatch response for all participants (needs to be transformed to function)
- Formatting dataframe as suitable model input

## Imports

In [1]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import ipywidgets as widgets
from IPython.display import display

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
#import eegyolk
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

In [2]:
metadata = pd.read_csv('metadata.csv', sep = ',')

In [3]:
metadata.head()

,eeg_file,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,105a,105,a,f,17,f,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,105a_epo.fif
1,107a,107,a,f,16,m,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,107a_epo.fif
2,106a,106,a,m,19,f,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,106a_epo.fif
3,109a,109,a,m,21,m,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,109a_epo.fif
4,110a,110,a,m,17,m,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,110a_epo.fif


## Read all epochs from files

The function below loads all filtered epochs from the dataframe which contains the metadata and the epochs filepaths + filenames. The epochs are arrays for each stimuli with a time interval of -0.2 to 0.8.

In [4]:
epochs = initialization_functions.read_filtered_data(metadata)

Checking out file: 105a_epo.fif
Checking out file: 107a_epo.fif
Checking out file: 106a_epo.fif
Checking out file: 109a_epo.fif
Checking out file: 110a_epo.fif
Checking out file: 112a_epo.fif
Checking out file: 111a_epo.fif
Checking out file: 114a_epo.fif
Checking out file: 115a_epo.fif
Checking out file: 117a_epo.fif
Checking out file: 116a_epo.fif
Checking out file: 118a_epo.fif
Checking out file: 119a_epo.fif
Checking out file: 123a_epo.fif
Checking out file: 122a_epo.fif
Checking out file: 124a_epo.fif
Checking out file: 127a_epo.fif
Checking out file: 125a_epo.fif
Checking out file: 126a_epo.fif
Checking out file: 130a_epo.fif
Checking out file: 128a_epo.fif
Checking out file: 129a_epo.fif
Checking out file: 131a_epo.fif
Checking out file: 135a_epo.fif
Checking out file: 133a_epo.fif
Checking out file: 137a_epo.fif
Checking out file: 138a_epo.fif
Checking out file: 139a_epo.fif
Checking out file: 141a_epo.fif
Checking out file: 144a_epo.fif
Checking out file: 143a_epo.fif
Checking

In [5]:
len(epochs)

101

## Create pandas dataframe with the average difference between standard and deviant responses

The function below needs `metadata`, the loaded `epochs` and the definition of the standard and deviant events as input. You should define your standard and deviant events as an array. In the function `input_mmr_prep` it's important to know that the assumption is made that the deviant follows after a standard event. Therefore the deviant belonging to the standard is the  standard event number + 1. Make sure your events are numbered like this, else the function won't calculate the mismatch response.  

In [6]:
def input_mmr_prep(metadata, epochs, standard_events, deviant_events): 
    # create dataframe with expected columns 
    df = pd.DataFrame(columns=["eeg_file",  "channel", "mean"]) # "paradigm",

    # loop over all participants
    for i in range(len(metadata['eeg_file'])):
        std_evoked = epochs[i][standard_events].average() 
        dev_evoked = epochs[i][deviant_events].average()

        # calculate the mismatch response between standard and deviant evoked
        evoked_diff = mne.combine_evoked([std_evoked, dev_evoked], weights=[1, -1])
        
        # get a list of all channels
        chnames_list = evoked_diff.info['ch_names']
        
        # compute for every channel the features of the mismatch line
        for channel in chnames_list: 
            chnames = mne.pick_channels(evoked_diff.info['ch_names'], include=[channel])
            roi_dict = dict(left_ROI=chnames) # combine_channels only takes a dictionary as input
            roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
            mmr = roi_evoked.to_data_frame()
            mmr_avg = mmr['left_ROI'].mean()
            mmr_std = mmr['left_ROI'].std()
            mmr_skew = mmr['left_ROI'].skew()
            mmr_var = mmr['left_ROI'].var()
            mmr_kurt = mmr['left_ROI'].kurtosis()
            
            df = df.append({'eeg_file': metadata['eeg_file'][i], 'channel': channel, 'mean' :  mmr_avg, 'std' : mmr_std, 'skew' : mmr_skew, 'kurt' : mmr_kurt, 'var' : mmr_var}, ignore_index=True) # add 'paradigm : paradigm' if we want to separate the paradigms 
    return df

In [7]:
# define the events for standard and deviant
standard_events = ['GiepM_S'] #'GiepM_S','GiepS_S','GopM_S','GopS_S'
deviant_events = ['GiepM_D'] #'GiepM_D','GiepS_D','GopM_D','GopS_D'


df = input_mmr_prep(metadata, epochs, standard_events, deviant_events)

/tmp/ipykernel_1981297/1378168468.py:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x7f49ac283b80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
/tmp/ipykernel_1981297/1378168468.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'eeg_file': metadata['eeg_file'][i], 'channel': channel, 'mean' :  mmr_avg, 'std' : mmr_std, 'skew' : mmr_skew, 'kurt' : mmr_kurt, 'var' : mmr_var}, ignore_index=True) # add 'paradigm : paradigm' if we want to separate the paradigms
/tmp/ipykernel_1981297/1378168468.py:20: RuntimeWarning: Less than 2 channels in group "left_ROI" when combining by method "functools.partial(<function mean at 0x7f49ac283b80>, axis=0)".
  roi_evoked = mne.channels.combine_channels(evoked_diff, roi_dict, method='mean')
/tmp/ipykernel_1981297/1378168468.p

In [8]:
df

,eeg_file,channel,mean,kurt,skew,std,var
0,105a,Fp1,-1.604101,-0.579970,-0.204490,4.845076,23.474765
1,105a,AF3,-0.635366,-0.747951,0.037597,2.613211,6.828872
2,105a,F7,9.750743,-0.369217,0.728353,8.701422,75.714750
3,105a,F3,4.175895,-0.539577,0.341130,3.411949,11.641396
4,105a,FC1,-5.082290,-0.982026,0.456234,3.506047,12.292363
...,...,...,...,...,...,...,...
3227,221a,F8,-0.284510,-0.670098,0.360154,2.162727,4.677388
3228,221a,AF4,-2.266951,-1.350999,0.138909,1.984651,3.938840
3229,221a,Fp2,-0.349263,-0.257129,-0.353276,1.857752,3.451242
3230,221a,Fz,-3.432783,-0.382503,0.341238,2.450541,6.005150


In [9]:
df = df.drop_duplicates(subset=['eeg_file','channel']) # ,'paradigm'

## Transpose dataframe into combination of paradigm and channel per participant

We now want a single row for every participant containing the paradigm and corresponding channels. The code below generates this dataframe. 

In [10]:
# transformation of the dataframe
df = df.pivot(index='eeg_file', columns=['channel']) # 'paradigm',

In [11]:
df.columns = ['_'.join(str(s).strip() for s in col if s) for col in df.columns]

In [12]:
df.reset_index(inplace=True)

## Merge and safe dataframe

We still need to merge some of the metadata into the dataframe, so we have the information of the age, gender and label of the participant. 

In [13]:
df = pd.merge(df, metadata, on='eeg_file')

In [14]:
pd.set_option('display.max_columns', None)

In [15]:
df

,eeg_file,mean_AF3,mean_AF4,mean_C3,mean_C4,mean_CP1,mean_CP2,mean_CP5,mean_CP6,mean_Cz,mean_F3,mean_F4,mean_F7,mean_F8,mean_FC1,mean_FC2,mean_FC5,mean_FC6,mean_Fp1,mean_Fp2,mean_Fz,mean_O1,mean_O2,mean_Oz,mean_P3,mean_P4,mean_P7,mean_P8,mean_PO3,mean_PO4,mean_Pz,mean_T7,mean_T8,kurt_AF3,kurt_AF4,kurt_C3,kurt_C4,kurt_CP1,kurt_CP2,kurt_CP5,kurt_CP6,kurt_Cz,kurt_F3,kurt_F4,kurt_F7,kurt_F8,kurt_FC1,kurt_FC2,kurt_FC5,kurt_FC6,kurt_Fp1,kurt_Fp2,kurt_Fz,kurt_O1,kurt_O2,kurt_Oz,kurt_P3,kurt_P4,kurt_P7,kurt_P8,kurt_PO3,kurt_PO4,kurt_Pz,kurt_T7,kurt_T8,skew_AF3,skew_AF4,skew_C3,skew_C4,skew_CP1,skew_CP2,skew_CP5,skew_CP6,skew_Cz,skew_F3,skew_F4,skew_F7,skew_F8,skew_FC1,skew_FC2,skew_FC5,skew_FC6,skew_Fp1,skew_Fp2,skew_Fz,skew_O1,skew_O2,skew_Oz,skew_P3,skew_P4,skew_P7,skew_P8,skew_PO3,skew_PO4,skew_Pz,skew_T7,skew_T8,std_AF3,std_AF4,std_C3,std_C4,std_CP1,std_CP2,std_CP5,std_CP6,std_Cz,std_F3,std_F4,std_F7,std_F8,std_FC1,std_FC2,std_FC5,std_FC6,std_Fp1,std_Fp2,std_Fz,std_O1,std_O2,std_Oz,std_P3,std_P4,std_P7,std_P8,std_PO3,std_PO4,std_Pz,std_T7,std_T8,var_AF3,var_AF4,var_C3,var_C4,var_CP1,var_CP2,var_CP5,var_CP6,var_Cz,var_F3,var_F4,var_F7,var_F8,var_FC1,var_FC2,var_FC5,var_FC6,var_Fp1,var_Fp2,var_Fz,var_O1,var_O2,var_Oz,var_P3,var_P4,var_P7,var_P8,var_PO3,var_PO4,var_Pz,var_T7,var_T8,ParticipantID,test,sex,age_months,dyslexic_parent,Group_AccToParents,path_eeg,path_epoch,path_eventmarkers,epoch_file
0,101a,0.497100,-0.397703,0.548694,-4.062833,-0.450172,-2.903579,1.669830,-4.953419,-0.373465,1.304724,-0.201433,-0.967906,-1.442023,1.055506,-2.251093,0.725231,-2.109742,-0.167942,-1.813575,0.050150,-2.145144,-3.081774,-4.896625,0.946727,-3.621544,1.219302,-0.507626,-1.189729,-0.245661,-1.910472,-0.534315,-2.323885,-0.281752,-0.963475,-0.484992,-1.086954,-1.029950,0.681150,-0.139426,-0.731020,-0.465958,-0.774395,-0.098127,0.405994,-0.903424,-0.775178,-1.184516,-0.775460,-1.386140,-0.215878,-0.636069,-0.382010,-0.085076,0.086099,0.517987,-0.872464,0.418126,-0.314737,-0.887824,-0.299761,-0.554054,-0.019716,-0.669277,-0.527265,0.564567,0.034387,-0.124069,0.073592,0.188272,0.877137,0.033243,0.113796,-0.106028,0.284948,0.259384,-0.387640,-0.052502,0.025826,-0.001928,0.068830,-0.104059,0.634468,-0.143197,0.126680,0.066457,0.710692,0.905066,0.044457,0.522454,0.098829,-0.071051,0.088000,-0.264271,0.515573,-0.197930,-0.525120,2.985758,2.594379,0.710128,2.899412,1.003934,1.808080,1.762945,3.748063,0.909117,2.248526,1.612332,5.456124,2.947768,1.130729,1.751691,1.771343,3.059773,3.118422,2.736698,2.100994,2.717166,2.370969,3.197210,2.156820,2.485309,2.843263,2.799955,2.050321,1.792446,1.546082,1.926054,3.025931,8.914748,6.730802,0.504282,8.406590,1.007883,3.269154,3.107976,14.047973,0.826494,5.055868,2.599613,29.769284,8.689334,1.278549,3.068422,3.137655,9.362212,9.724553,7.489513,4.414175,7.382991,5.621493,10.222152,4.651872,6.176762,8.084143,7.839750,4.203818,3.212862,2.390369,3.709684,9.156260,101,a,m,20,m,At risk,../../volume-ceph/ePodium_projectfolder/dataset,../../volume-ceph/nadine_storage/processed_epochs,../../volume-ceph/ePodium_projectfolder/events,101a_epo.fif
1,103a,-2.544724,-3.113051,-1.181946,2.656960,2.329480,4.669067,-3.759660,6.810110,1.792349,-1.926158,-3.026434,-1.497297,-3.051092,1.350173,0.056760,-1.467667,0.391760,-2.651314,1.773268,-2.587891,24.513732,-5.152285,5.100410,1.819144,4.847754,17.547848,7.665762,6.115613,7.102235,6.524891,-1.050560,3.969248,-0.188597,-0.342524,-0.871011,-1.010533,-1.188359,-1.388351,2.107175,-1.443581,-1.120354,-0.558567,-0.393680,-0.300297,-0.309336,-1.266313,-1.034464,-0.413853,-0.810543,-0.259105,-0.744564,-0.881908,-1.666530,0.380136,-1.233465,-0.962455,-1.373557,-1.572636,-1.164217,-1.314515,-1.204992,-1.347161,-0.537895,-1.143331,-0.207216,0.087872,0.070365,-0.487317,-0.084918,-0.277000,-0.872701,-0.270746,-0.218889,-0.278572,0.042117,-0.419571,0.206954,0.197833,-0.459381,-0.399501,-0.463544,-0.162412,0.048490,-0.327720,-0.068889,-0.502710,0.198078,0.143789,-0.178979,-0.175128,-0.474649,-0.359

Drop some unnecessary columns. 

In [16]:
df = df.drop(['eeg_file',
       'dyslexic_parent', 'path_eeg','path_epoch',
       'epoch_file', 'path_eventmarkers'], axis =1)

In [17]:
df['sex'] = np.where(
    (df['sex']=='m'), 1,0)

df['Group_AccToParents'] = np.where(
    (df['Group_AccToParents']=='At risk'), 1,0)

In [18]:
first = df.pop('Group_AccToParents')
df.insert(0, 'Group_AccToParents', first)

In [19]:
df.to_csv('df_avg_mmr.csv', index=False) # safe dataframe

## PCA analysis on feature reduction 

In [20]:
X = df.drop('Group_AccToParents',1)
y = df['Group_AccToParents']

/tmp/ipykernel_1175280/1933137523.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df.drop('Group_AccToParents',1)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [22]:
pca = PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

ValueError: could not convert string to float: 'a'

In [23]:
explained_variance = pca.explained_variance_ratio_

AttributeError: 'PCA' object has no attribute 'explained_variance_ratio_'

In [ ]:
explained_variance

In [ ]:
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

In [ ]:
X.shape